In [ ]:
### Code to extract all tables from rating manual ###

import re
import pandas as pd
from io import StringIO
import fitz  # PyMuPDF

def pdf_to_text(pdf_path):
    """
    Opens a PDF file with PyMuPDF (fitz) and extracts all text content.
    Returns the full text as a single string.
    """
    try:
        pdf_doc = fitz.open(pdf_path)
    except Exception as e:
        print(f"Error opening PDF file: {e}")
        return ""
    
    full_text = []
    for page_num in range(len(pdf_doc)):
        page = pdf_doc[page_num]
        text = page.get_text("text")
        full_text.append(text)
    pdf_doc.close()
    return "\n".join(full_text)

def extract_tables(text):
    """
    Extracts blocks of lines assumed to be tables from the text.
    A new table block is assumed to start when a line consists only of
    uppercase letters, spaces, slashes, or dashes (a header line).
    Returns a dictionary mapping table names to a block of text (the table).
    """
    lines = text.splitlines()
    tables = {}  # maps table name -> text block
    current_table_lines = []
    current_table_name = None
    header_pattern = re.compile(r'^[A-Z0-9 /-]{3,}$')
    
    for line in lines:
        if not line.strip():
            continue
        
        if header_pattern.match(line.strip()):
            # If we have a table in progress, save it.
            if current_table_lines and current_table_name:
                tables[current_table_name] = "\n".join(current_table_lines)
                current_table_lines = []
            # Use the header (truncated to 31 characters) as the table name.
            current_table_name = line.strip()[:31]
        else:
            # Add lines that contain at least one digit.
            if re.search(r'\d', line):
                current_table_lines.append(line)
            else:
                # You might process or log lines that don't match.
                pass
    # Save the final table block if present.
    if current_table_lines and current_table_name:
        tables[current_table_name] = "\n".join(current_table_lines)
    return tables

def table_text_to_df(table_text):
    """
    Attempts to convert a block of text into a pandas DataFrame.
    First tries read_csv with whitespace delimiter; if that fails,
    it falls back to read_fwf for fixed width formatted data.
    """
    df = pd.DataFrame()
    try:
        # Attempt to read using read_csv; skip malformed lines.
        df = pd.read_csv(StringIO(table_text),
                         delim_whitespace=True,
                         engine='python',
                         on_bad_lines='skip')
    except Exception as e:
        print("Error reading table block with read_csv:", e)
        try:
            df = pd.read_fwf(StringIO(table_text))
        except Exception as e2:
            print("Error reading table block with read_fwf:", e2)
            df = pd.DataFrame()
    return df

def sanitize_sheet_name(name):
    """
    Sanitizes a sheet name for Excel by replacing invalid characters
    (:, \, /, ?, *, [, ]) with an underscore. Excel sheet names are
    limited to 31 characters.
    """
    return re.sub(r'[:\\/*?\[\]]', '_', name)[:31]

# ----- Jupyter Notebook Processing -----
# Specify your PDF file path here:
pdf_file_path = "Filings/Initial/2025-02-10/KYP64967.pdf"

# Convert the PDF to text.
full_text = pdf_to_text(pdf_file_path)
if not full_text:
    print("No text was extracted from the PDF.")
else:
    # Extract table blocks from the text.
    table_blocks = extract_tables(full_text)
    
    # Process each table block into a DataFrame.
    dataframes = {}
    for table_name, block_text in table_blocks.items():
        df = table_text_to_df(block_text)
        # Drop rows that are completely NaN.
        df.dropna(how="all", inplace=True)
        if not df.empty:
            dataframes[table_name] = df
    
    # Write all DataFrames to a single Excel workbook with one sheet per table.
    output_filename = "extracted_tables.xlsx"
    with pd.ExcelWriter(output_filename) as writer:
        for sheet_name, df in dataframes.items():
            safe_sheet_name = sanitize_sheet_name(sheet_name)
            try:
                df.to_excel(writer, sheet_name=safe_sheet_name, index=False)
            except Exception as e:
                print(f"Error writing sheet '{safe_sheet_name}': {e}")
    
    print(f"Extracted {len(dataframes)} tables. Excel file saved as '{output_filename}'")
